# 

# Convert *.raw files using command line options

1) <b>-d</b> flag sets directory to search for Thermo RawFileReader dll's
2) <b>-n</b> flag specifies the number of threads to use
3) <b>-sf</b> list of terms to search for in each scan header/filter. Scans that contain these words are ommited
4) <b>-o</b> path to folder where the converted files will be saved

In [ ]:
!python3.9 raw_to_parquet.py /Users/n.t.wamsley/RIS_temp/MATRIX_MATCHED_CAL_CURVES/ -d ./libs -n 1 -o /Users/n.t.wamsley/RIS_temp/MATRIX_MATCHED_CAL_CURVES/

PATH
/Users/n.t.wamsley/Projects/ThermoRawFileToParquetConverter/libs/ThermoFisher.CommonCore.Data.dll
PATH
/Users/n.t.wamsley/Projects/ThermoRawFileToParquetConverter/libs/ThermoFisher.CommonCore.RawFileReader.dll
Raw Files Directory: /Users/n.t.wamsley/RIS_temp/MATRIX_MATCHED_CAL_CURVES/
Thermo dlls Directory:  ./libs
Scan Filter Regex List:  ['ITMS']
Scan Header Used:  False
Number of Workers:  1
Parquet File Output Folder:  /Users/n.t.wamsley/RIS_temp/MATRIX_MATCHED_CAL_CURVES/
PATH
/Users/n.t.wamsley/Projects/ThermoRawFileToParquetConverter/libs/ThermoFisher.CommonCore.Data.dll
PATH
/Users/n.t.wamsley/Projects/ThermoRawFileToParquetConverter/libs/ThermoFisher.CommonCore.RawFileReader.dll
Processing raw file:  MOUSE_SIL_400FMOL_FAIMS_5uLLoop_gas403_voltage2150_fixedbugs_042724_01
Reading Scans...
 51%|█████████████████▉                 | 39221/76536 [00:14<00:17, 2088.83it/s]

In [1]:
import json
json_args_f = open('args.json')
json_args = json.load(json_args_f)
display(json_args)

{'raw_dir': '../raw',
 'thermo_dlls': '../libs',
 'scan_filter_regex_list': ['ITMS', 'kazoo'],
 'num_workers': 12,
 'parquet_out': './parquet_out'}

# Convert *.raw files using .json arguments
In the previous example running the raw file converter using command line options generated a "args.json"
file specifying the options. Rather than supplying options through flags in the command line, a simple .json
file can be specified. 

In [ ]:
!python3 raw_to_parquet.py args.json

In [4]:
import argparse
import sys
import os 
from os import listdir
from os.path import isfile, join
import time
from collections import defaultdict
import clr #From pythonnet package
#Paths to thermo dlls
import os
from os.path import abspath
thermo_data_path = abspath(os.path.join("./libs/ThermoFisher.CommonCore.Data.dll"))
thermo_rawfilereader_path = abspath(os.path.join("./libs/ThermoFisher.CommonCore.RawFileReader.dll"))

In [6]:
for path in [thermo_data_path, thermo_rawfilereader_path]:
    if not os.path.isfile(path):
        raise Exception("The file: " + path + " does not exist!")
    else:
        print("PATH")
        print(path)
        clr.AddReference(path)

PATH
/Users/n.t.wamsley/Projects/ThermoRawFileToParquetConverter/libs/ThermoFisher.CommonCore.Data.dll
PATH
/Users/n.t.wamsley/Projects/ThermoRawFileToParquetConverter/libs/ThermoFisher.CommonCore.RawFileReader.dll


In [7]:
from ThermoFisher.CommonCore.RawFileReader import RawFileReaderAdapter
from ThermoFisher.CommonCore.Data.Business import Device, Scan
from ThermoFisher.CommonCore.Data.Interfaces import IScanFilter
from ThermoFisher.CommonCore.Data.FilterEnums import MassAnalyzerType, MSOrderType

In [14]:
raw_file_paths = ['/Users/n.t.wamsley/TEST_DATA/PXD046444/20230324_OLEP08_800ng_30min_E5H50Y45_180K_2Th3p5ms_01.raw']

In [10]:
import struct
import pyarrow as pa
import pyarrow.parquet as pq
import pyarrow.csv as csv
import pyarrow.dataset as dataset
import gc
import numpy as np


In [11]:
 scans = Scans()

NameError: name 'Scans' is not defined

In [17]:
rawFile = RawFileReaderAdapter.FileFactory(raw_file_paths[0])
rawFile.SelectInstrument(Device.MS, 1)


In [18]:
    first_scan_number = rawFile.RunHeaderEx.FirstSpectrum
    last_scan_number = rawFile.RunHeaderEx.LastSpectrum

In [23]:
test_stream = rawFile.GetCentroidStream(200000, False)

In [26]:
len(np.array(test_stream.Masses))

2570

In [30]:
test_data = rawFile.GetSimplifiedScan(200000)

In [35]:
x = test_data.ToString()

In [37]:
scan_number = 200000

In [40]:
        trailers = rawFile.GetTrailerExtraValues(scan_number)
        trailer_labels = rawFile.GetTrailerExtraInformation(scan_number)
        charge_state = 0

In [44]:
{'key': index_to_header_key_map[i].split(":")[0], 
                                        'value': str(rawFile.GetTrailerExtraValue(scan_number, i)).strip()}

NameError: name 'index_to_header_key_map' is not defined

In [46]:
test_data = rawFile.GetScanEventForScanNumber(scan_number)

In [48]:
scan_data = test_data.get_ScanData()

In [54]:
a = rawFile.GetScanEventForScanNumber(scan_number).GetReaction(0)

In [55]:
a.PrecursorMass

651.54602

In [60]:
index_to_header_key_map = defaultdict(str)


if True:
    logEntry = rawFile.GetTrailerExtraHeaderInformation()
    t = 0
    for i in logEntry:
        if t >0:
            index_to_header_key_map[t] = i.Label
        t +=1

In [62]:
scan_header = []
for i in index_to_header_key_map.keys():
    scan_header.append({'key': index_to_header_key_map[i].split(":")[0], 
                            'value': str(rawFile.GetTrailerExtraValue(scan_number, i)).strip()}) # TODO: Parse into correct datatype for each scan header. There are 40 of them


In [66]:
scan_stats = rawFile.GetScanStatsForScanNumber(scan_number)

In [70]:
a = scan_stats.ToString()

In [134]:
file_header = rawFile.GetScanEventForScanNumber(200000)

In [90]:
cid = file_header.get_HigherEnergyCiD().ToString()

In [91]:
file_header.get_HigherEnergyCiDValue()

25.0

In [97]:
file_header.get_ScanMode().ToString()

'Full'

In [99]:
a = file_header.get_ScanMode()

In [102]:
file_header.get_MSOrder().ToString()

'Ms2'

In [118]:
scan_data = file_header.get_ScanData()

In [131]:
file_header.GetFirstPrecursorMass(0)

0.0

In [128]:
file_header.GetIsolationWidth(0)

2.0009096

In [132]:
file_header.GetMass(0)

651.54602

In [139]:
test_range = file_header.GetMassRange(0)

In [140]:
test_range.get_Low()

150.0

In [141]:
test_range.get_High()

1351.19482421875

In [143]:
scan_stats = rawFile.GetScanStatsForScanNumber(scan_number)

In [146]:
file_header.GetEnergy(0)

25.0